In [4]:
!python3 generate_teams.py
!python3 generate_timeslots.py
!python3 spikeball_groups.py 5

/opt/homebrew/bin/python3: can't open file '/Users/aksseet/Spikeball/spikeball_group': [Errno 2] No such file or directory
